In [1]:
import keras
from keras.applications import vgg16
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
#output = keras.layers.Dense(12, activation='softmax')(model.layers[-2].output)
#resnet_model = keras.models.Model(model.inputs, output)

In [3]:
cd train

/Users/pranjalgupta/Documents/Domain Adaption 2020/train


In [4]:
input_data = []
target_data = []

In [5]:
def preprocess_image(img):
    pil_image = keras.preprocessing.image.load_img(img, target_size=(224,224))
    np_image = keras.preprocessing.image.img_to_array(pil_image)
    #final_img = np.expand_dims(np_image,axis=0)
    return np_image

In [6]:
img_paths = open('image_list.txt','r')
n = 0
p = 0
for i in img_paths.readlines():
    a = i.split()
    if a[1] == str(p):
        if n<200:
            input_data.append(preprocess_image(a[0]))
            target = np.array(p).reshape(-1)
            onehot = np.eye(12)[target]
            target_data.append(onehot)
            n = n+1
        else:
            p = p+1
            n = 0   

In [7]:
np_input_data = np.array(input_data)
np_target_data = np.array(target_data)
print(np_input_data.shape)
print(np_target_data.shape)

(2400, 224, 224, 3)
(2400, 1, 12)


In [8]:
np_target_data.resize(2400,12)

In [9]:
print(np_input_data.shape)
print(np_target_data.shape)

(2400, 224, 224, 3)
(2400, 12)


In [10]:
processed_input = vgg16.preprocess_input(np_input_data.copy())

In [11]:
mid_pred_train = model.predict(processed_input)


In [12]:
mid_pred = np.reshape(mid_pred_train, (2400,7*7*512))
print(mid_pred.shape)

(2400, 25088)


In [20]:
input_val_data = []
target_val_data = []
img_validation_paths = open('image_list.txt','r')
n = 0
p = 0
for i in img_validation_paths.readlines():
    a = i.split()
    if a[1] == str(p):
        if n<200:
            input_val_data.append(preprocess_image(a[0]))
            target = np.array(p).reshape(-1)
            onehot = np.eye(12)[target]
            target_val_data.append(onehot)
            n = n+1
        else:
            p = p+1
            n = 0   

In [21]:
np_input_val_data = np.array(input_val_data)
np_target_val_data = np.array(target_val_data)
np_target_val_data = np.reshape(np_target_val_data, (2400,12))
print(np_input_val_data.shape)
print(np_target_val_data.shape)

(2400, 224, 224, 3)
(2400, 12)


In [22]:
proc_val_input = vgg16.preprocess_input(np_input_val_data.copy())
mid_pred_val = model.predict(proc_val_input)

In [23]:
mid_pred_3 = np.reshape(mid_pred_val,(2400,7*7*512))

In [13]:
#classifier model on top of the trained model. 
vgg_model = keras.models.Sequential()
vgg_model.add(keras.layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
vgg_model.add(keras.layers.Dropout(0.5))
vgg_model.add(keras.layers.Dense(12, activation='softmax'))

vgg_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 6,425,868
Trainable params: 6,425,868
Non-trainable params: 0
_________________________________________________________________


In [14]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
vgg_model.fit(mid_pred, np_target_data, epochs=10)


Epoch 1/10
2400/2400 [==============================] - 3s 1ms/step - loss: 3.0272 - accuracy: 0.8808
Epoch 2/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.4312 - accuracy: 0.9663
Epoch 3/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.2030 - accuracy: 0.9792
Epoch 4/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.1238 - accuracy: 0.9867
Epoch 5/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.2588 - accuracy: 0.9808
Epoch 6/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.2318 - accuracy: 0.9817
Epoch 7/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.3462 - accuracy: 0.9783
Epoch 8/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.2371 - accuracy: 0.9858
Epoch 9/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.4590 - accuracy: 0.9804
Epoch 10/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.163

In [16]:
vgg_model.save('vgg_classifier.h5')

In [61]:
train_result = vgg_model.evaluate(mid_pred, np_target_data)

2400/2400 [==============================] - 1s 250us/step


In [67]:
val_result = vgg_model.evaluate(mid_pred_3, np_target_val_data)

2400/2400 [==============================] - 0s 154us/step


In [63]:
def preprocess_tstinput(img):
    tst_image = keras.preprocessing.image.load_img(img, target_size=(224,224))
    tstnp_image = keras.preprocessing.image.img_to_array(tst_image)
    s = np.expand_dims(tstnp_image, axis=0)
    s = vgg16.preprocess_input(s.copy()) 
    f = model.predict(s)
    f = np.reshape(f, (1,7*7*512))
    return f

In [65]:
image_path = 'person/person_1203954.jpg'
pred = vgg_model.predict(preprocess_tstinput(image_path))
print(pred)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [68]:
print('Training Accuracy (synthetic image) - {:.2f}'.format(train_result[1]*100))
#print('Test Accuracy (synthetic image) - {:.2f}'.format(tst_result[1]*100))
print('Validation (real image) Accuracy - {:.2f}'.format(val_result[1]*100))

Training Accuracy (synthetic image) - 16.67
Validation (real image) Accuracy - 11.08


## Code for Domain Adaption 

In [27]:
from keras import backend as K

In [28]:
'''import random
def create_pairs():
    pairs = []
    labels = []
    for i in range(12):
        for j in range(200):
            pairs = pairs+ [[np_input_data[(200*i)+j],np_input_val_data[(200*i)+j]]]
            x = random.randrange(1,12)
            i_n = (i+x)%12
            pairs = pairs + [[np_input_data[(200*i)+j], np_input_val_data[(200*i_n)+j]]]
            labels = labels + [1,0]
    return np.array(pairs),np.array(labels)'''

'import random\ndef create_pairs():\n    pairs = []\n    labels = []\n    for i in range(12):\n        for j in range(200):\n            pairs = pairs+ [[np_input_data[(200*i)+j],np_input_val_data[(200*i)+j]]]\n            x = random.randrange(1,12)\n            i_n = (i+x)%12\n            pairs = pairs + [[np_input_data[(200*i)+j], np_input_val_data[(200*i_n)+j]]]\n            labels = labels + [1,0]\n    return np.array(pairs),np.array(labels)'

In [29]:
import random
def create_pairs():
    pairs = []
    labels = []
    for i in range(12):
        for j in range(200):
            pairs = pairs+ [[mid_pred[(200*i)+j],mid_pred_3[(200*i)+j]]]
            x = random.randrange(1,12)
            i_n = (i+x)%12
            pairs = pairs + [[mid_pred[(200*i)+j], mid_pred_3[(200*i_n)+j]]]
            labels = labels + [1,0]
    return np.array(pairs),np.array(labels)

In [30]:
train_pairs, train_labels =  create_pairs()

In [35]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [36]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [37]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [60]:
'''def basnet_out(input_dataset):
    processed_input_dataset = keras.layers.Lambda(lambda x:vgg16.preprocess_input(x))(input_dataset)
    middle_prediction = model(processed_input_dataset)
    middle_prediction = keras.layers.Lambda(lambda x:tf.reshape(x, [-1, 7*7*512]))(middle_prediction)
    encoding_dataset = vgg_model(middle_prediction)    
    return encoding_dataset'''

In [38]:
def basnet_out(input_dataset):
    encoding_dataset = vgg_model(input_dataset)    
    return encoding_dataset

In [39]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [47]:
input_a = keras.layers.Input(shape = (25088,))
input_b = keras.layers.Input(shape = (25088,))

In [50]:
processed_a = basnet_out(input_a)
processed_b = basnet_out(input_b)

In [52]:
distance = keras.layers.Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

In [69]:
da_model = keras.models.Model([input_a,input_b],distance)

In [70]:
da_model.compile(optimizer = keras.optimizers.RMSprop(), loss = contrastive_loss, metrics = [accuracy])

In [71]:
da_model.fit([train_pairs[:,0,:], train_pairs[:,1,:]], train_labels , epochs = 10)

Epoch 1/10
4800/4800 [==============================] - 11s 2ms/step - loss: 0.5638 - accuracy: 0.5277
Epoch 2/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5378 - accuracy: 0.5202
Epoch 3/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5579 - accuracy: 0.5240
Epoch 4/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5748 - accuracy: 0.5454
Epoch 5/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5900 - accuracy: 0.5565
Epoch 6/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5775 - accuracy: 0.5638
Epoch 7/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5903 - accuracy: 0.5550
Epoch 8/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5487 - accuracy: 0.5969
Epoch 9/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.5291 - accuracy: 0.6083
Epoch 10/10
4800/4800 [==============================] - 9s 2ms/step - loss: 0.47

In [72]:
da_model.save('final_da_model.h5')